In [ ]:
%%sh
pip install -q --upgrade pip
pip install -q --upgrade diffusers transformers scipy ftfy huggingface_hub roboflow

In [ ]:
from huggingface_hub import notebook_login

# Required to get access to stable diffusion model
notebook_login()

In [ ]:
import torch
from diffusers import StableDiffusionPipeline

pipeline = StableDiffusionPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5", torch_dtype=torch.float16, revision="fp16"
)

pipeline = pipeline.to("cuda")

In [ ]:
import os

from IPython.display import Image, display


def generate_images(
    prompt,
    num_images_to_generate,
    num_images_per_prompt=4,
    guidance_scale=8,
    output_dir="images",
    display_images=False,
):

    num_iterations = num_images_to_generate // num_images_per_prompt
    os.makedirs(output_dir, exist_ok=True)

    for i in range(num_iterations):
        images = pipeline(
            prompt, num_images_per_prompt=num_images_per_prompt, guidance_scale=guidance_scale
        )
        for idx, image in enumerate(images.images):
            image_name = f"{output_dir}/image_{(i*num_images_per_prompt)+idx}.png"
            image.save(image_name)
            if display_images:
                display(Image(filename=image_name, width=128, height=128))

In [ ]:
# 1000 images takes a couple hours on a SageMaker Studio Lab GPU instance 

generate_images("human on a forklift", 1000, guidance_scale=4, display_images=True)



In [ ]:
import os
HOME = os.getcwd()
image_dir = os.path.join(HOME, "images", "")

In [ ]:
file_path = HOME/images/

In [ ]:
from roboflow import Roboflow
import glob
import os

## DEFINITIONS
# glob params
image_dir = os.path.join(HOME, "images", "")
file_extension_type = ".png"

## INIT
# roboflow pip params
rf = Roboflow(api_key="YOUR_ROBOFLOW_API_KEY")
upload_project = rf.workspace().project("YOUR_ROBOFLOW_PROJECT")

## MAIN
# glob images
image_glob = glob.glob(image_dir + '/*' + file_extension_type)

# perform upload
for image in image_glob:
    upload_project.upload(image, num_retry_uploads=3)
    print("*** Processing image [" + str(len(image_glob)) + "] - " + image + " ***")

In [ ]:
# Future Enhancement
#1. Generate images
#2. Use Roboflow Universe model for auto label generation 
#3. Upload images with label confidence greater than 50% to a project via API
#4. Review and hand annotate the rest in Roboflow 